In [1]:
import visa
import time
import numpy as np

In [5]:
rm = visa.ResourceManager()
rm.list_resources()

('ASRL3::INSTR', 'ASRL10::INSTR', 'GPIB0::12::INSTR', 'GPIB0::14::INSTR')

In [6]:
ametek7270 = rm.open_resource('USB0::0x0A2D::0x001B::09140123::RAW')
keithley6221 = rm.open_resource('GPIB0::14::INSTR')
guass = rm.open_resource('GPIB0::12::INSTR')

In [18]:
def get_position():
    f = guass.query("RDGFIELD?")
    f = round(float(f[:-2]) / 10) * 10
    return f

In [20]:
get_position()

930

In [6]:
SENSITIVITIES = [
        0.0, 2.0e-9, 5.0e-9, 10.0e-9, 20.0e-9, 50.0e-9, 100.0e-9,
        200.0e-9, 500.0e-9, 1.0e-6, 2.0e-6, 5.0e-6, 10.0e-6,
        20.0e-6, 50.0e-6, 100.0e-6, 200.0e-6, 500.0e-6, 1.0e-3,
        2.0e-3, 5.0e-3, 10.0e-3, 20.0e-3, 50.0e-3, 100.0e-3,
        200.0e-3, 500.0e-3, 1.0
    ]
TIME_CONSTANTS = [
        10.0e-6, 20.0e-6, 50.0e-6, 100.0e-6, 200.0e-6, 500.0e-6,
        1.0e-3, 2.0e-3, 5.0e-3, 10.0e-3, 20.0e-3, 50.0e-3, 100.0e-3,
        200.0e-3, 500.0e-3, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0,
        100.0, 200.0, 500.0, 1.0e3, 2.0e3, 5.0e3, 10.0e3,
        20.0e3, 50.0e3, 100.0e3
    ]

In [62]:
def init_ac(ac, freq=157, start_I=0.002, stand_by_I=0.002):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:AMPL {}".format(start_I))
    ac.write("SOUR:WAVE:FREQ {}".format(freq))
    ac.write("SOUR:WAVE:ARM")
    ac.write("SOUR:WAVE:INIT")
    time.sleep(3)
    ac.write("SOUR:WAVE:AMPL {}".format(stand_by_I))
    

In [10]:
def init_ametek(ame, time_constant=0.5, sensitivity=10.0e-6, harmonic=1):
    ame.write("TC {}".format(TIME_CONSTANTS.index(time_constant)))
    ame.write("SEN {}".format(SENSITIVITIES.index(sensitivity)))
    ame.write("REFN {}".format(harmonic))

In [21]:
def lockin_measurement(ame, time_per_point=0.1, points_stored=10, show_data=False):
    X = []
    Y = []
    M = []
    P = []
    
    for i in range(points_stored):
        time.sleep(time_per_point)
        m, p = ame.query('MP.').replace("\n\x00", "").split(",")
        x, y = ame.query('XY.').replace("\n\x00", "").split(",")
        X.append(float(x))
        Y.append(float(y))
        M.append(float(m))
        P.append(float(p))
    res = []
    all_result = [X, Y, M, P]
    for i in all_result:
        i = np.array(i)
        res.append(i.mean())
        res.append(i.std())
    if show_data:
        print(all_result)
    return res

In [23]:
lockin_measurement(ametek7270, time_per_point=0.2, points_stored=10, show_data=False)

[-8.653e-07,
 3.90000000000001e-09,
 1.037e-07,
 1.951922129594313e-09,
 8.711e-07,
 3.884584919910994e-09,
 173.157,
 0.12165936051122148]

In [24]:
def field_measurement(ame, sample_id="TIG_PT_Rxy_2nd", ac_A=0.002,
                        time_per_point=0.1, points_stored=10, counts_per_angle=10, wait_time_out=10,
                        wait_per_measurement=5, freq=153):
    filename = sample_id + "_ac_{}_duration_{}_f_{}.csv".format(ac_A, "on", freq)
    with open(filename, 'w') as f:
        f.write("X,dX,Y,dY,M,dM,P,dP,Field\n")
    point_count = 0
    count = 0 # if position didn't change count++, stop measurement once count > 
    last_position = None
    while count < wait_time_out:
        pos = get_position()
        print("current field: {}".format(pos))
        if pos == last_position and point_count > counts_per_angle:
            count += 1
            time.sleep(1)
        else:
            res = lockin_measurement(ame, time_per_point=time_per_point, points_stored=points_stored)
            res.append(pos)
            res = [str(i) for i in res]
            with open(filename, 'a') as f:
                f.write(",".join(res) + "\n")
            time.sleep(wait_per_measurement)
            if pos == last_position:
                point_count += 1
            else:
                point_count =0
            last_position = pos
            count = 0
                

In [25]:
field_measurement(ametek7270, sample_id="TIG_PT_Rxy_2nd_test", ac_A=0.002,
                        time_per_point=0.1, points_stored=10, counts_per_angle=10, wait_time_out=100,
                        wait_per_measurement=5, freq=153)

current field: 1040


current field: 1040


current field: 1040


current field: 1040


current field: 1040


...

current field: 520


current field: 520


current field: 520


current field: 520


PermissionError: [Errno 13] Permission denied: 'TIG_PT_Rxy_2nd_ac_0.002_duration_on_f_153.csv'

In [11]:
fileds = [5000, 1000]
steps = [500, 10]

In [19]:
def generate_fields(fileds, steps):
    assert (len(fileds) == len(steps))
    res = []
    for i in range(len(fileds)):
        res.extend([f for f in range(fileds[i], fileds[i + 1] if i+1 < len(fileds) else 0, 
                                   -1 * steps[i])])
    res = res + [0] + [-1* f for f in res[::-1]]
    res += res[::-1][1:]
    return res  

In [1]:
generate_fields(fileds, steps)

NameError: name 'generate_fields' is not defined

In [ ]:
def field_depend(vsm, ame, keith, frequency, current, points_stored, fields, 
                 ramp_rate, time_per_point=0.1, sample_id="test"):
    
    init_ac(keith, freq=frequency, start_I=current, stand_by_I=current)
    filename = sample_id + "_ac_{}_duration_{}_f_{}.csv".format(current, frequency)
    
    with open(filename, 'w') as f:
        f.write("X,dX,Y,dY,M,dM,P,dP\n")
        
    for f in fields:
        set_and_wait(f, ramp_rate)
        res = lockin_always_on(ame, time_per_point=time_per_point, 
                         points_stored=points_stored, show_data=False)
        
        res = [str(i) for i in res]
        with open(filename, 'a') as f:
            f.write(",".join(res) + "\n")


In [ ]:
generate_fields(fileds, steps)